# Wines Points prediction 

In [2]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

In [3]:
#pip install countryinfo

In [4]:
#pip install forex_python

In [5]:
import pandas as pd
import cufflinks as cf; cf.go_offline()
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
import re
import numpy as np
import string
import emoji
import re as regex
import nltk
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\or.nagari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\or.nagari\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
wine_reviews_analysis = pd.read_csv("C:/Users/or.nagari/Python_ML_2023/Exercises/Final Project/wine_reviews_analysis.csv")
wine_reviews_analysis.rename({'Unnamed: 0':'id'}, axis=1,inplace=True)
wine_reviews_analysis.shape

(119677, 10)

In [8]:
#pd.set_option('display.max_colwidth', 300)

In [9]:

target = 'points'
test_df, train_df = train_test_split(wine_reviews_analysis, test_size=0.8, random_state=42)
train_df.dropna(axis=0, inplace=True)
test_df.dropna(axis=0, inplace=True)

train_df.shape, test_df.shape

((95742, 10), (23935, 10))

In [10]:
X_train, y_train = train_df.drop(target, axis=1), train_df[[target]]
X_test, y_test = test_df.drop(target, axis=1), test_df[[target]]

X_train.head(1)

,country,price,taster_name,province,region_1,year,winery,cleaned_descriptions,variety
94591,France,18.0,Roger Voss,Loire Valley,Coteaux du Giennois,2014,Domaine de Villargeau,herbal cool wine Concentrated citrus flavors p...,Sauvignon Blanc


#Baselines

In [11]:
# #train and test for baseline
train_data = wine_reviews_analysis.sample(frac = 0.8)
test_data = wine_reviews_analysis[~wine_reviews_analysis.index.isin(train_data.index)]
assert(len(train_data) + len(test_data) == len(wine_reviews_analysis))

In [12]:
def calc_prediction_quality(df, pred_score_col, true_score_col):
    return pd.Series({'MSE': mean_squared_error(df[true_score_col], df[pred_score_col]),
                      'MAE': mean_absolute_error(df[true_score_col], df[pred_score_col]),
                      'R2': r2_score(df[true_score_col], df[pred_score_col])})

In [13]:
test_data['basiline_1_predicted_points'] = train_data.points.mean()
b1_stats = calc_prediction_quality(test_data, 'basiline_1_predicted_points', 'points')
b1_stats

MSE    9.554984
MAE    2.537486
R2    -0.000019
dtype: float64

In [14]:
avg_points_by_country = train_data.groupby('country').points.mean()
avg_points_by_country.head()

country
Argentina    86.688316
Australia    88.552961
Austria      90.090834
Brazil       84.447368
Bulgaria     87.913043
Name: points, dtype: float64

In [15]:
test_data['basiline_2_predicted_points'] = test_data.country.map(avg_points_by_country).fillna(train_data.points.mean())
b2_stats = calc_prediction_quality(test_data, 'basiline_2_predicted_points', 'points')
b2_stats

MSE    9.036182
MAE    2.462820
R2     0.054279
dtype: float64

In [16]:
avg_points_by_country_and_region = train_data.groupby(['country','province']).points.mean().rename('basiline_3_predicted_points')
avg_points_by_country_and_region.head()

country    province        
Argentina  Mendoza Province    86.809053
           Other               85.960298
Australia  Australia Other     85.478261
           New South Wales     87.846154
           South Australia     89.021804
Name: basiline_3_predicted_points, dtype: float64

In [17]:
test_data_with_baseline_3 = test_data.merge(avg_points_by_country_and_region, on = ['country','province'], how='left')
test_data_with_baseline_3.basiline_3_predicted_points = test_data_with_baseline_3.basiline_3_predicted_points.fillna(test_data_with_baseline_3.basiline_2_predicted_points).fillna(test_data.basiline_1_predicted_points)
test_data_with_baseline_3.shape, test_data.shape

((23935, 13), (23935, 12))

In [18]:
b3_stats = calc_prediction_quality(test_data_with_baseline_3, 'basiline_3_predicted_points', 'points')
b3_stats

MSE    8.531853
MAE    2.375124
R2     0.107061
dtype: float64

In [19]:
baseline_summary = pd.DataFrame([b1_stats, b2_stats, b3_stats], index=['baseline_1', 'baseline_2','baseline_3'])
baseline_summary

,MSE,MAE,R2
baseline_1,9.554984,2.537486,-0.000019
baseline_2,9.036182,2.462820,0.054279
baseline_3,8.531853,2.375124,0.107061


In [20]:
baseline_summary

,MSE,MAE,R2
baseline_1,9.554984,2.537486,-0.000019
baseline_2,9.036182,2.462820,0.054279
baseline_3,8.531853,2.375124,0.107061


In [21]:
#wine_reviews[wine_reviews.description.apply(lambda d: 'easydrinking' in d)].description.values

In [22]:
#first try

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the KNeighborsRegressorTransformer class
class KNeighborsRegressorTransformer(KNeighborsRegressor, TransformerMixin):
    def transform(self, X):
        return self.predict(X)

# Define categorical and numerical features
categorical_features = ['country', 'province', 'region_1', 'taster_name', 'variety']
numerical_features = ['price', 'year']
des_feature = 'cleaned_descriptions'

# Create transformers for categorical, numerical, and text features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer())
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features),
        ('text', text_transformer, des_feature)
    ])



In [24]:
# Create the pipeline for KNN
knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressorTransformer())
])

In [25]:
# Create the pipeline for Linear Regression
linear_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [26]:
# Create the pipeline for Random Forest
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])


In [27]:
# Fit the KNN pipeline on the training data
knn_pipeline.fit(X_train, y_train)

# Make predictions using the KNN pipeline
knn_predictions = knn_pipeline.predict(X_test)

# Calculate evaluation metrics for KNN
knn_mse = mean_squared_error(y_test, knn_predictions)
knn_mae = mean_absolute_error(y_test, knn_predictions)
knn_r2 = r2_score(y_test, knn_predictions)
print("KNN - MSE: {:.2f}, MAE: {:.2f}, R2: {:.2f}".format(knn_mse, knn_mae, knn_r2))


KNN - MSE: 4.32, MAE: 1.61, R2: 0.55


In [28]:
new_record = pd.DataFrame({"MSE": knn_mse, "MAE": knn_mae, "R2": knn_r2}, index=["KNN"])
baseline_summary = baseline_summary.append(new_record)

In [29]:
baseline_summary

,MSE,MAE,R2
baseline_1,9.554984,2.537486,-0.000019
baseline_2,9.036182,2.462820,0.054279
baseline_3,8.531853,2.375124,0.107061
KNN,4.315334,1.609426,0.546611


In [30]:
#KNN	4.315334	1.609426	0.546611

In [31]:

# Fit the Linear Regression pipeline on the training data
linear_pipeline.fit(X_train, y_train)

# Make predictions using the Linear Regression pipeline
linear_predictions = linear_pipeline.predict(X_test)

# Calculate evaluation metrics for Linear Regression
linear_mse = mean_squared_error(y_test, linear_predictions)
linear_mae = mean_absolute_error(y_test, linear_predictions)
linear_r2 = r2_score(y_test, linear_predictions)

print("Linear Regression - MSE: {:.2f}, MAE: {:.2f}, R2: {:.2f}".format(linear_mse, linear_mae, linear_r2))


Linear Regression - MSE: 3.39, MAE: 1.39, R2: 0.64


In [32]:
#Linear Regression - MSE: 3.39, MAE: 1.39, R2: 0.64

In [33]:
new_record = pd.DataFrame({"MSE": linear_mse, "MAE": linear_mae, "R2": linear_r2}, index=["Linear Regression"])
baseline_summary = baseline_summary.append(new_record)

In [34]:
# Fit the Random Forest pipeline on the training data
rf_pipeline.fit(X_train, y_train)

# Make predictions using the Random Forest pipeline
rf_predictions = rf_pipeline.predict(X_test)

# Calculate evaluation metrics for Random Forest
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

# Print the evaluation metrics for each model
print("Random Forest - MSE: {:.2f}, MAE: {:.2f}, R2: {:.2f}".format(rf_mse, rf_mae, rf_r2))


Random Forest - MSE: 3.21, MAE: 1.39, R2: 0.66


In [35]:
#Random Forest - MSE: 3.22, MAE: 1.39, R2: 0.66

In [41]:
baseline_summary['MSE'] = baseline_summary['MSE'].round(2)
baseline_summary['MAE'] = baseline_summary['MAE'].round(2)
baseline_summary['R2'] = baseline_summary['R2'].round(2)

In [42]:
baseline_summary

,MSE,MAE,R2
baseline_1,9.55,2.54,-0.00
baseline_2,9.04,2.46,0.05
baseline_3,8.53,2.38,0.11
KNN,4.32,1.61,0.55
Linear Regression,3.39,1.39,0.64
Random Forest,3.21,1.39,0.66


In [44]:
summary = baseline_summary

In [45]:
summary

,MSE,MAE,R2
baseline_1,9.55,2.54,-0.00
baseline_2,9.04,2.46,0.05
baseline_3,8.53,2.38,0.11
KNN,4.32,1.61,0.55
Linear Regression,3.39,1.39,0.64
Random Forest,3.21,1.39,0.66


In [46]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [47]:


# Preprocessing
X = wine_reviews_analysis['cleaned_descriptions'].copy()
y = wine_reviews_analysis['points'].copy()

# Tokenize and convert textual features to numerical sequences
max_words = 1000  # Maximum number of words to consider
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_text1 = tokenizer.texts_to_sequences(X)

max_seq_length = max(len(seq) for seq in X_text1)

X_text1 = pad_sequences(X_text1, maxlen=max_seq_length)

# Load pre-trained GloVe embeddings
embedding_path = "C:/Users/or.nagari/Python_ML_2023/glove100.txt"

embedding_dim = 100  # Adjust the embedding dimension based on the pre-trained embedding

embedding_index = {}
with open(embedding_path, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = embedding

# Create word-to-index mapping
word_index = tokenizer.word_index

# Prepare the embedding matrix
num_words = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Split the data into training and testing sets
X_train_text1, X_test_text1, y_train, y_test = train_test_split(X_text1, y, test_size=0.2, random_state=42)

# Define the model
text_input_1 = Input(shape=(max_seq_length,))
embedding_1 = Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False)(text_input_1)
flatten_1 = Flatten()(embedding_1)
x = Dense(64, activation='relu')(flatten_1)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='linear')(x)

model = Model(inputs=[text_input_1], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# Train the model
model.fit([X_train_text1], y_train,
          batch_size=32,
          epochs=10,
          validation_data=([X_test_text1], y_test))

# Evaluate the model
predictions = model.predict([X_test_text1])

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print('MSE:', mse)
print('MAE:', mae)
print('R2 Score:', r2)


Epoch 1/10
2992/2992 [==============================] - 38s 12ms/step - loss: 90.8032 - val_loss: 34.6336
Epoch 2/10
2992/2992 [==============================] - 42s 14ms/step - loss: 30.6907 - val_loss: 30.6457
Epoch 3/10
2992/2992 [==============================] - 41s 14ms/step - loss: 23.7535 - val_loss: 26.7931
Epoch 4/10
2992/2992 [==============================] - 37s 12ms/step - loss: 18.4464 - val_loss: 19.6573
Epoch 5/10
2992/2992 [==============================] - 37s 12ms/step - loss: 14.7612 - val_loss: 19.4890
Epoch 6/10
2992/2992 [==============================] - 37s 13ms/step - loss: 11.9814 - val_loss: 15.3946
Epoch 7/10
2992/2992 [==============================] - 44s 15ms/step - loss: 10.0274 - val_loss: 14.6053
Epoch 8/10
2992/2992 [==============================] - 39s 13ms/step - loss: 8.5994 - val_loss: 11.3949
Epoch 9/10
2992/2992 [==============================] - 39s 13ms/step - loss: 7.4116 - val_loss: 10.5182
Epoch 10/10
748/748 [===========================

In [48]:
new_record = pd.DataFrame({"MSE": mse, "MAE": mae, "R2": r2}, index=["pre-trained-Glove"])
summary = summary.append(new_record)

In [49]:
# Assuming you have a word_index and embedding_index

missing_words = []
total_words = len(word_index)

# Iterate over word_index and check for embeddings
for word, i in word_index.items():
    if word not in embedding_index:
        missing_words.append(word)

num_missing_words = len(missing_words)
percentage_missing = (num_missing_words / total_words) * 100

print("Percentage of words without embeddings:", round(percentage_missing),'%')

Percentage of words without embeddings: 45 %


In [50]:
missing_words[:10]

['fullbodied',
 'mediumbodied',
 'midpalate',
 'offdry',
 'finegrained',
 'blackfruit',
 'winerys',
 'easydrinking',
 'screwcap',
 'lightbodied']

In [51]:




# Preprocessing
X = wine_reviews_analysis['cleaned_descriptions'].copy()
y = wine_reviews_analysis['points'].copy()

# Tokenize and convert textual features to numerical sequences
max_words = 1000  # Maximum number of words to consider
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_text1 = tokenizer.texts_to_sequences(X)

max_seq_length = max(len(seq) for seq in X_text1)

X_text1 = pad_sequences(X_text1, maxlen=max_seq_length)

# Split the data into training and testing sets
X_train_text1, X_test_text1, y_train, y_test = train_test_split(X_text1, y, test_size=0.2, random_state=42)

# Define the model
text_input_1 = Input(shape=(max_seq_length,))
embedding_dim = 16
embedding_1 = Embedding(input_dim=max_words, output_dim=embedding_dim)(text_input_1)
flatten_1 = Flatten()(embedding_1)
x = Dense(64, activation='relu')(flatten_1)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='linear')(x)

model = Model(inputs=[text_input_1], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# Train the model
model.fit([X_train_text1], y_train,
          batch_size=32,
          epochs=10,
          validation_data=([X_test_text1], y_test))

# Evaluate the model
predictions = model.predict([X_test_text1])

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print('MSE:', mse)
print('MAE:', mae)
print('R2 Score:', r2)


Epoch 1/10
2992/2992 [==============================] - 16s 5ms/step - loss: 120.1820 - val_loss: 4.1832
Epoch 2/10
2992/2992 [==============================] - 14s 5ms/step - loss: 4.0598 - val_loss: 3.8193
Epoch 3/10
2992/2992 [==============================] - 15s 5ms/step - loss: 3.9468 - val_loss: 3.9019
Epoch 4/10
2992/2992 [==============================] - 13s 4ms/step - loss: 3.8699 - val_loss: 4.2136
Epoch 5/10
2992/2992 [==============================] - 13s 4ms/step - loss: 3.7320 - val_loss: 3.8141
Epoch 6/10
2992/2992 [==============================] - 13s 4ms/step - loss: 3.7091 - val_loss: 4.0497
Epoch 7/10
2992/2992 [==============================] - 14s 5ms/step - loss: 3.6290 - val_loss: 4.0448
Epoch 8/10
2992/2992 [==============================] - 15s 5ms/step - loss: 3.5881 - val_loss: 3.9269
Epoch 9/10
2992/2992 [==============================] - 13s 4ms/step - loss: 3.6084 - val_loss: 6.4989
Epoch 10/10
748/748 [==============================] - 2s 2ms/step
MSE:

In [52]:
new_record = pd.DataFrame({"MSE": mse, "MAE": mae, "R2": r2}, index=["not_trained"])
summary = summary.append(new_record)

In [55]:
summary['MSE'] = summary['MSE'].round(2)
summary['MAE'] = summary['MAE'].round(2)
summary['R2'] = summary['R2'].round(2)

In [56]:
summary

,MSE,MAE,R2
baseline_1,9.55,2.54,-0.00
baseline_2,9.04,2.46,0.05
baseline_3,8.53,2.38,0.11
KNN,4.32,1.61,0.55
Linear Regression,3.39,1.39,0.64
Random Forest,3.21,1.39,0.66
pre-trained-Glove,9.58,2.45,-0.01
not_trained,3.91,1.56,0.59
